In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import roc_auc_score

import seaborn as sns
from sklearn.datasets import fetch_openml

In [2]:
# --- 1. Загружаем данные
try:
  df = sns.load_dataset("titanic")[['age', 'embarked', 'fare', 'sex', 'class', 'survived']]
  df = df.dropna(subset=['fare', 'sex', 'class', 'survived'])  # удалим строки с критическими пропусками
except:
  !gdown 1iLZLfioJK9kLlhXURCS_9nqalp5IEGFG
  df = pd.read_csv('/content/short_titanic.csv')


In [3]:
# --- Метод 1: Удаление строк с пропусками
print(df.shape, df['survived'].mean())
df1 = df.dropna()
print(df1.shape, df1['survived'].mean())
print(f'всего удалили {df.shape[0] - df1.shape[0]}')
X1 = pd.get_dummies(df1.drop(columns='survived'), drop_first=True)
y1 = df1['survived']
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, random_state=0)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
rocauc_dropna = roc_auc_score(y_test, y_pred)


(891, 6) 0.3838383838383838
(712, 6) 0.4044943820224719
всего удалили 179


In [4]:
# --- Метод 2: Простая импутация (медиана и "Missing")
df2 = df.copy()
df2['age'] = df2['age'].fillna(df2['age'].median())
df2['embarked'] = df2['embarked'].fillna('Missing')
X2 = pd.get_dummies(df2.drop(columns='survived'), drop_first=True)
y2 = df2['survived']
X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.2, random_state=0)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
rocauc_simple = roc_auc_score(y_test, y_pred)

In [5]:
# --- Метод 3: Импутация + флаги пропусков
df3 = df.copy()
df3['age_missing'] = df3['age'].isna().astype(int)
df3['embarked_missing'] = df3['embarked'].isna().astype(int)
df3['age'] = df3['age'].fillna(df3['age'].median())
df3['embarked'] = df3['embarked'].fillna('Missing')
X3 = pd.get_dummies(df3.drop(columns='survived'), drop_first=True)
y3 = df3['survived']
X_train, X_test, y_train, y_test = train_test_split(X3, y3, test_size=0.2, random_state=0)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
rocauc_flagged = roc_auc_score(y_test, y_pred)

In [6]:
# --- Метод 4: Импутация с помощью линейной регрессии

df4 = df.copy()

# Для регрессии возраста нужно one-hot кодировать категориальные признаки
# Сначала заполняем пропуски в embarked для удобства
df4['embarked'] = df4['embarked'].fillna('Missing')

# Делим на части с известным и неизвестным возрастом
df_age_train = df4[df4['age'].notna()]
df_age_pred = df4[df4['age'].isna()]

# One-hot кодирование для признаков, используемых в регрессии
X_age_train = pd.get_dummies(df_age_train[['fare', 'sex', 'class', 'embarked']], drop_first=True)
y_age_train = df_age_train['age']

X_age_pred = pd.get_dummies(df_age_pred[['fare', 'sex', 'class', 'embarked']], drop_first=True)

# Выровнять колонки у X_age_pred относительно X_age_train (если есть несоответствия)
X_age_pred = X_age_pred.reindex(columns=X_age_train.columns, fill_value=0)

# Обучаем линейную регрессию
linreg = LinearRegression()
linreg.fit(X_age_train, y_age_train)

# Предсказываем пропущенный возраст
predicted_ages = linreg.predict(X_age_pred)
df4.loc[df4['age'].isna(), 'age'] = predicted_ages

# Теперь готовим данные для классификации
X4 = pd.get_dummies(df4.drop(columns='survived'), drop_first=True)
y4 = df4['survived']

X_train, X_test, y_train, y_test = train_test_split(X4, y4, test_size=0.2, random_state=0)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
rocauc_predicted = roc_auc_score(y_test, y_pred)

In [7]:
# --- Итоговые результаты
print("Метод 1 (удаление строк):          Roc Auc =", round(rocauc_dropna, 3))
print("Метод 2 (простая импутация):       Roc Auc =", round(rocauc_simple, 3))
print("Метод 3 (импутация + флаги):       Roc Auc =", round(rocauc_flagged, 3))
print("Метод 4 (импутация линейной регрессией): Roc Auc =", round(rocauc_predicted, 3))


Метод 1 (удаление строк):          Roc Auc = 0.806
Метод 2 (простая импутация):       Roc Auc = 0.781
Метод 3 (импутация + флаги):       Roc Auc = 0.792
Метод 4 (импутация линейной регрессией): Roc Auc = 0.778
